In [3]:
import csv
import random
import math
from collections import Counter, defaultdict
import itertools

In [4]:
filename = 'D:/University/Data Analysis & Machine Learning 4 course/data_lab2,3/mushroom/agaricus-lepiota.data'

# Load data
data = []
with open(filename, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if len(row) == 0:
            continue
        data.append(row)

In [5]:
print("Number of instances loaded:", len(data))
print("Example rows:")
for i in range(5):
    print(data[i])

Number of instances loaded: 8124
Example rows:
['p', 'x', 's', 'n', 't', 'p', 'f', 'c', 'n', 'k', 'e', 'e', 's', 's', 'w', 'w', 'p', 'w', 'o', 'p', 'k', 's', 'u']
['e', 'x', 's', 'y', 't', 'a', 'f', 'c', 'b', 'k', 'e', 'c', 's', 's', 'w', 'w', 'p', 'w', 'o', 'p', 'n', 'n', 'g']
['e', 'b', 's', 'w', 't', 'l', 'f', 'c', 'b', 'n', 'e', 'c', 's', 's', 'w', 'w', 'p', 'w', 'o', 'p', 'n', 'n', 'm']
['p', 'x', 'y', 'w', 't', 'p', 'f', 'c', 'n', 'n', 'e', 'e', 's', 's', 'w', 'w', 'p', 'w', 'o', 'p', 'k', 's', 'u']
['e', 'x', 's', 'g', 'f', 'n', 'f', 'w', 'b', 'k', 't', 'e', 's', 's', 'w', 'w', 'p', 'w', 'o', 'e', 'n', 'a', 'g']


In [6]:
data = [row for row in data if '?' not in row]

In [7]:
classes = [row[0] for row in data]
features = [row[1:] for row in data]

In [8]:
class_map = {'e':0, 'p':1}
y = [class_map[c] for c in classes]

In [9]:
unique_values_per_feature = []
for col_idx in range(len(features[0])):
    unique_vals = sorted(list(set([row[col_idx] for row in features])))
    unique_values_per_feature.append(unique_vals)

In [10]:
for i, values in enumerate(unique_values_per_feature):
    print(f"Feature {i}: {values}")

Feature 0: ['b', 'c', 'f', 'k', 's', 'x']
Feature 1: ['f', 'g', 's', 'y']
Feature 2: ['b', 'c', 'e', 'g', 'n', 'p', 'w', 'y']
Feature 3: ['f', 't']
Feature 4: ['a', 'c', 'f', 'l', 'm', 'n', 'p']
Feature 5: ['a', 'f']
Feature 6: ['c', 'w']
Feature 7: ['b', 'n']
Feature 8: ['g', 'h', 'k', 'n', 'p', 'r', 'u', 'w', 'y']
Feature 9: ['e', 't']
Feature 10: ['b', 'c', 'e', 'r']
Feature 11: ['f', 'k', 's', 'y']
Feature 12: ['f', 'k', 's', 'y']
Feature 13: ['b', 'c', 'g', 'n', 'p', 'w', 'y']
Feature 14: ['b', 'c', 'g', 'n', 'p', 'w', 'y']
Feature 15: ['p']
Feature 16: ['w', 'y']
Feature 17: ['n', 'o', 't']
Feature 18: ['e', 'l', 'n', 'p']
Feature 19: ['h', 'k', 'n', 'r', 'u', 'w']
Feature 20: ['a', 'c', 'n', 's', 'v', 'y']
Feature 21: ['d', 'g', 'l', 'm', 'p', 'u']


In [11]:
X = []
for row in features:
    encoded = []
    for col_idx, val in enumerate(row):
        val_index = unique_values_per_feature[col_idx].index(val)
        encoded.append(val_index)
    X.append(encoded)
     
X = [list(r) for r in X]

In [12]:
random.seed(42)
indices = list(range(len(X)))
random.shuffle(indices)
split_point = int(0.8 * len(X))

train_idx = indices[:split_point]
val_idx = indices[split_point:]

X_train = [X[i] for i in train_idx]
y_train = [y[i] for i in train_idx]
X_val = [X[i] for i in val_idx]
y_val = [y[i] for i in val_idx]

In [13]:
def chi_square_feature(X, y, feature_index):
    # Extract the column
    col = [x[feature_index] for x in X]
    # Possible values for this feature
    vals = set(col)
    # Classes
    cls = set(y)

    # Create contingency table
    # counts[val][class] = count
    counts = defaultdict(lambda: defaultdict(int))
    val_counts = defaultdict(int)
    cls_counts = defaultdict(int)

    for val_i, c in zip(col, y):
        counts[val_i][c] += 1
        val_counts[val_i] += 1
        cls_counts[c] += 1

    total = len(X)

    # Compute chi-square
    chi2 = 0.0
    for v in val_counts:
        for c in cls_counts:
            observed = counts[v][c]
            expected = (val_counts[v] * cls_counts[c]) / total
            if expected > 0:
                chi2 += (observed - expected)**2 / expected

    return chi2

In [14]:
chi2_scores = []
for f_idx in range(len(X_train[0])):
    score = chi_square_feature(X_train, y_train, f_idx)
    chi2_scores.append((f_idx, score))

# Sort features by chi2 score descending
chi2_scores.sort(key=lambda x: x[1], reverse=True)

In [15]:
print("Top 10 features by Chi-Squared score:")
for f_idx, score in chi2_scores[:10]:
    print(f"Feature {f_idx}: {score}")

Top 10 features by Chi-Squared score:
Feature 4: 4216.231587517972
Feature 19: 3066.755544614285
Feature 18: 2368.9379968954386
Feature 11: 2273.2428804854344
Feature 12: 2177.760791727101
Feature 13: 1571.7964100729591
Feature 9: 1562.7992228158053
Feature 14: 1461.0214147272734
Feature 8: 1224.2251270707259
Feature 2: 1027.012391729265


In [16]:
class KNN:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X, y):
        self.X = X
        self.y = y

    def predict_one(self, x):
        # Compute distances
        distances = []
        for i, x_train in enumerate(self.X):
            # Euclidean distance
            dist = 0
            for a,b in zip(x_train, x):
                dist += (a-b)**2
            dist = math.sqrt(dist)
            distances.append((dist, self.y[i]))
        distances.sort(key=lambda d: d[0])
        neighbors = [c for (_, c) in distances[:self.k]]
        # Majority vote
        most_common = Counter(neighbors).most_common(1)[0][0]
        return most_common

    def predict(self, X):
        return [self.predict_one(x) for x in X]

In [17]:
def accuracy(y_true, y_pred):
    correct = sum(int(a==b) for a,b in zip(y_true, y_pred))
    return correct / len(y_true)

In [18]:
def forward_selection(X_train, y_train, X_val, y_val, max_features=None):
    available_features = set(range(len(X_train[0])))
    selected_features = []
    best_acc = 0.0
    last_improvement = True

    # We can stop either when no improvement or if we reached max_features
    while last_improvement and (max_features is None or len(selected_features) < max_features):
        last_improvement = False
        candidate_feature = None

        for f in available_features:
            trial_features = selected_features + [f]
            # Extract these features from training and validation
            X_train_sub = [[x[i] for i in trial_features] for x in X_train]
            X_val_sub = [[x[i] for i in trial_features] for x in X_val]

            model = KNN(k=5)
            model.fit(X_train_sub, y_train)
            y_val_pred = model.predict(X_val_sub)
            val_acc = accuracy(y_val, y_val_pred)
            if val_acc > best_acc:
                best_acc = val_acc
                candidate_feature = f

        if candidate_feature is not None:
            selected_features.append(candidate_feature)
            available_features.remove(candidate_feature)
            last_improvement = True

    return selected_features

In [19]:
wrapper_selected_features = forward_selection(X_train, y_train, X_val, y_val)

In [20]:
def evaluate_top_n_features_chi2(n):
    top_features = [f_idx for (f_idx, _) in chi2_scores[:n]]
    X_train_sub = [[x[i] for i in top_features] for x in X_train]
    X_val_sub = [[x[i] for i in top_features] for x in X_val]

    model = KNN(k=5)
    model.fit(X_train_sub, y_train)
    y_val_pred = model.predict(X_val_sub)
    return accuracy(y_val, y_val_pred)

In [21]:
best_chi2_n = None
best_chi2_acc = 0.0
# Let's just try from 1 to all features:
for n in range(1, len(X_train[0]) + 1):
    val_acc = evaluate_top_n_features_chi2(n)
    if val_acc > best_chi2_acc:
        best_chi2_acc = val_acc
        best_chi2_n = n

chi2_selected_features = [f_idx for (f_idx, _) in chi2_scores[:best_chi2_n]]

In [22]:
model_all = KNN(k=5)
model_all.fit(X_train, y_train)
y_val_pred_all = model_all.predict(X_val)
acc_all = accuracy(y_val, y_val_pred_all)

# Filter method features
X_train_chi2 = [[x[i] for i in chi2_selected_features] for x in X_train]
X_val_chi2 = [[x[i] for i in chi2_selected_features] for x in X_val]
model_chi2 = KNN(k=5)
model_chi2.fit(X_train_chi2, y_train)
y_val_pred_chi2 = model_chi2.predict(X_val_chi2)
acc_chi2 = accuracy(y_val, y_val_pred_chi2)

# Wrapper method features
X_train_wrapper = [[x[i] for i in wrapper_selected_features] for x in X_train]
X_val_wrapper = [[x[i] for i in wrapper_selected_features] for x in X_val]
model_wrapper = KNN(k=5)
model_wrapper.fit(X_train_wrapper, y_train)
y_val_pred_wrapper = model_wrapper.predict(X_val_wrapper)
acc_wrapper = accuracy(y_val, y_val_pred_wrapper)

In [23]:
print("Number of features selected by filter method:", best_chi2_n)
print("Number of features selected by wrapper method:", len(wrapper_selected_features))

print("Validation Accuracy (All features):", acc_all)
print("Validation Accuracy (Filter method):", acc_chi2)
print("Validation Accuracy (Wrapper method):", acc_wrapper)

Number of features selected by filter method: 10
Number of features selected by wrapper method: 3
Validation Accuracy (All features): 1.0
Validation Accuracy (Filter method): 1.0
Validation Accuracy (Wrapper method): 1.0
